In [ ]:
!pip install gast==0.2.2

In [2]:
import json
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.losses import sparse_categorical_crossentropy # used for integer targets
from tensorflow.keras.optimizers import Nadam
from random import shuffle, seed, sample
from tqdm import tqdm, tqdm_notebook
import numpy as np
import os

Using TensorFlow backend.


Finding all titles, training, and target sets composed by Chopin

## Parse Input file into title, train, target

In [3]:
titles_h = []
train_h = []
target_h = []
with open('data/Haydn_input.json', 'r') as handle: # EDIT WITH COMPOSER
    for i,line in enumerate(tqdm(handle)):
        song = json.loads(line)
        titles_h.append(song['title'])
        train_h.append(song['train'])
        target_h.append(song['target'])

40it [00:14,  2.70it/s]


In [2]:
titles = []
train = []
target = []
with open('data/Sonata_input.json', 'r') as handle: # EDIT WITH COMPOSER
    for i,line in enumerate(tqdm(handle)):
        song = json.loads(line)
        titles.append(song['title'])
        train.append(song['train'])
        target.append(song['target'])

407it [05:43,  1.19it/s]


In [8]:
len(set(titles))

271

## Below for Parsing Chopin

In [41]:
unique_chopin = []
keep_indices = []
for i,song in enumerate(titles):
    song = song.lower()
    if 'andante' in song:
        song = 'andante op 22'
    if 'ballade no. 2' in song:
        song = 'ballade no 2'
    if 'barcarolle' in song:
        song = 'barcarolle'
    if 'berceuse' in song:
        song = 'barceuse'
    if 'etude op. 10 no. 1' in song:
        song = 'etude op. 10 no. 1'
    if 'etude op. 10 no. 12' in song:
        song = 'etude op. 10 no. 12'
    if 'etude op. 10 no. 2' in song:
        song = 'etude op. 10 no. 2'
    if 'etude op. 10 no. 4' in song:
        song = 'etude op. 10 no. 4'
    if 'etude op. 10 no. 8' in song:
        song = 'etude op. 10 no. 8'
    if 'etude op. 25 no. 1' in song:
        song = 'etude op. 25 no. 1'
    if 'etude op. 25 no. 10' in song:
        song = 'etude op. 25 no. 10'
    if 'etude op. 25 no. 11' in song:
        song = 'etude op. 25 no. 11'
    if 'etude op. 25 no. 12' in song:
        song = 'etude op. 25 no. 12'
    if 'etude op. 25 no. 6' in song:
        song = 'etude op. 25 no. 6'
    if 'op. 49' in song:
        song = 'fantasy op. 49'
    if 'polonaise-fantasie' in song:
        song = 'polonaise-fantasie'
    if 'scherzo no. 2' in song:
        song = 'scherzo no. 2'
    if 'scherzo no. 3' in song:
        song = 'scherzo no. 3'
    if 'scherzo no. 4' in song:
        song = 'scherzo no. 4'
    if 'sonata no. 3' in song:
        song = 'sonata no. 3'
    if song not in unique_chopin:
        unique_chopin.append(song)
        keep_indices.append(i)

## Below for Parsing Brahms

In [96]:
unique_brahms = []
keep_indices = []
for i,song in enumerate(titles):
    song = song.lower()
    if 'sonata in' in song:
        song = 'sonata in f minor op 5'
    if song not in unique_brahms:
        unique_brahms.append(song)
        keep_indices.append(i)

## Below for Parsing Sonatas

In [3]:
unique_son = []
keep_indices = []
for i,song in enumerate(titles):
    if song not in unique_son:
        unique_son.append(song)
        keep_indices.append(i)

## Below for Parsing Hadyn

In [4]:
unique_ha = []
keep_indices = []
for i,song in enumerate(titles_h):
    if song not in unique_ha:
        unique_ha.append(song)
        keep_indices.append(i)

## Keeping Unique Songs

In [5]:
unique_train = []
for i,each in enumerate(train_h):
    if i in keep_indices:
        unique_train.append(each)

In [6]:
unique_target = []
for i, each in enumerate(target_h):
    if i in keep_indices:
        unique_target.append(each)

In [7]:
len(unique_train)

29

In [8]:
len(unique_target)

29

In [9]:
len(unique_ha)

29

## Modeling

In [10]:
def create_model(seq_len, unique_notes, dropout=0.3, output_emb=100, rnn_unit=128, dense_unit=64):
    inputs = tf.keras.layers.Input(shape=(seq_len,))
    embedding = tf.keras.layers.Embedding(input_dim=unique_notes, output_dim=output_emb, input_length=seq_len)(inputs)
    forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit, return_sequences=True))(embedding)
#     #forward_pass , att_vector = SeqSelfAttention(
#         return_attention=True,
#         attention_activation='sigmoid',
#         attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
#         attention_width=50,
#         kernel_regularizer=tf.keras.regularizers.l2(1e-4),
#         bias_regularizer=tf.keras.regularizers.l1(1e-4),
#         attention_regularizer_weight=1e-4,
#         )(forward_pass)
    forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
    forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit, return_sequences=True))(forward_pass)
#     forward_pass , att_vector2 = SeqSelfAttention(
#         return_attention=True,
#         attention_activation='sigmoid',
#         attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, # multiplicative
#         attention_width=50,
#         kernel_regularizer=tf.keras.regularizers.l2(1e-4),
#         bias_regularizer=tf.keras.regularizers.l1(1e-4),
#         attention_regularizer_weight=1e-4,
#         )(forward_pass)
    forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
    forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit))(forward_pass)
    forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
    forward_pass = tf.keras.layers.Dense(dense_unit)(forward_pass)
    forward_pass = tf.keras.layers.LeakyReLU()(forward_pass)
    outputs = tf.keras.layers.Dense(unique_notes, activation = "softmax")(forward_pass)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='generate_scores_rnn')
    return model

In [11]:
with open('data/Haydn_index.json', 'r') as read_file:
    dict_index = json.load(read_file)

In [12]:
unique_notes = len(dict_index)

In [24]:
unique_notes

16870

In [25]:
model = create_model(50, unique_notes)

In [26]:
model.summary()

Model: "generate_scores_rnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 50, 100)           1687000   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 50, 256)           175872    
_________________________________________________________________
dropout_6 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 50, 256)           295680    
_________________________________________________________________
dropout_7 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 256)       

In [27]:
optimizer = Nadam()

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 model=model)
checkpoint_dir = 'models/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
loss_fn = sparse_categorical_crossentropy

In [28]:
class TrainModel:
  
    def __init__(self, epochs, sample_train, sample_target, batch_nnet_size, batch_song, optimizer, checkpoint, loss_fn,
               checkpoint_prefix, total_songs, model):
        self.epochs = epochs
        self.sample_train = sample_train
        self.sample_target = sample_target
        self.batch_nnet_size = batch_nnet_size
        self.batch_song = batch_song
        self.optimizer = optimizer
        self.checkpoint = checkpoint
        self.loss_fn = loss_fn
        self.checkpoint_prefix = checkpoint_prefix
        self.total_songs = total_songs
        self.model = model
    
    def train(self):
        for epoch in tqdm_notebook(range(self.epochs),desc='epochs'):
            # for each epochs, we shuffle the list of all the datasets
            c = list(zip(self.sample_train, self.sample_target))
            shuffle(c)
            self.sample_train, self.sample_target = zip(*c)
            loss_total = 0
            steps = 0
            steps_nnet = 0

            # Iterate all songs by the length of sample input (total_songs) and batches (batch_song)
            for i in tqdm_notebook(range(0,self.total_songs, self.batch_song), desc='MUSIC'):
                # EXAMPLE: [0,5,10,15,20] FOR TOTAL_SONGS = 20 AND BATCH_SONG = 5
                steps += 1
                #inputs_nnet_large, outputs_nnet_large = generate_batch_song(
                 #   self.sample_input, self.batch_song, start_index=i, fs=self.frame_per_second, 
                  #  seq_len=seq_len, use_tqdm=False) # We use the function that have been defined here
                #inputs_nnet_large = np.array(self.note_tokenizer.transform(inputs_nnet_large), dtype=np.int32)
                #outputs_nnet_large = np.array(self.note_tokenizer.transform(outputs_nnet_large), dtype=np.int32)
                
                # EXAMPLE LARGE INPUTS = ARRAY([1,2,3,4],[2,3,4,5],[2,3,4,5],[2,3,4,5],[1,2,3,4])
                input_batch = [y for x in self.sample_train[i:i+self.batch_song] for y in x]
                output_batch = [y for x in self.sample_target[i:i+self.batch_song] for y in x]
                c = list(zip(input_batch, output_batch))
                all_sample = sample(c, 10000)
#                 start = c[0:5000]
#                 middle = int(len(c)/2)
#                 middle_left = c[middle:middle-5000]
#                 middle_right = c[middle:middle+5000]
#                 end = c[-5000:]
#                 all_sample = start + middle_left + middle_right + end
                input_batch, output_batch = zip(*all_sample)
                inputs_nnet_large = np.array(input_batch)
                outputs_nnet_large = np.array(output_batch)

                # Get an index of all windows in a song
                index_shuffled = np.arange(start=0, stop=len(inputs_nnet_large))
                np.random.shuffle(index_shuffled)
                
                for nnet_steps in tqdm_notebook(range(0,len(index_shuffled),self.batch_nnet_size)):
                    steps_nnet += 1
                    current_index = index_shuffled[nnet_steps:nnet_steps+self.batch_nnet_size]

                    inputs_nnet, outputs_nnet = inputs_nnet_large[current_index], outputs_nnet_large[current_index]

                    # To make sure no exception thrown by tensorflow on autograph
                    if len(inputs_nnet) // self.batch_nnet_size != 1:
                        break
                    loss = self.train_step(inputs_nnet, outputs_nnet)
                    loss_total += tf.math.reduce_sum(loss)
                    if steps_nnet % 20 == 0:
                        print("epochs {} | Steps {} | total loss : {}".format(epoch + 1, steps_nnet,loss_total))

                    #checkpoint.save(file_prefix = self.checkpoint_prefix)

    @tf.function
    def train_step(self, inputs, targets):
        with tf.GradientTape() as tape:
            prediction = self.model(inputs)
            loss = self.loss_fn(targets, prediction)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return loss

In [29]:
seq_len = 50
EPOCHS = 5
BATCH_SONG = 5
BATCH_NNET_SIZE = 96
TOTAL_SONGS = len(unique_target)
FRAME_PER_SECOND = 5

In [30]:
train_class = TrainModel(EPOCHS, unique_train, unique_target, BATCH_NNET_SIZE, BATCH_SONG, optimizer, checkpoint, loss_fn, checkpoint_prefix, 
                        TOTAL_SONGS, model)
train_class.train()

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:56: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


epochs 1 | Steps 20 | total loss : Tensor("add_3139:0", shape=(), dtype=float32)
epochs 1 | Steps 40 | total loss : Tensor("add_3159:0", shape=(), dtype=float32)
epochs 1 | Steps 60 | total loss : Tensor("add_3179:0", shape=(), dtype=float32)
epochs 1 | Steps 80 | total loss : Tensor("add_3199:0", shape=(), dtype=float32)
epochs 1 | Steps 100 | total loss : Tensor("add_3219:0", shape=(), dtype=float32)


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:56: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epochs 5 | Steps 120 | total loss : Tensor("add_5734:0", shape=(), dtype=float32)
epochs 5 | Steps 140 | total loss : Tensor("add_5754:0", shape=(), dtype=float32)
epochs 5 | Steps 160 | total loss : Tensor("add_5774:0", shape=(), dtype=float32)
epochs 5 | Steps 180 | total loss : Tensor("add_5794:0", shape=(), dtype=float32)
epochs 5 | Steps 200 | total loss : Tensor("add_5814:0", shape=(), dtype=float32)


epochs 5 | Steps 220 | total loss : Tensor("add_5833:0", shape=(), dtype=float32)
epochs 5 | Steps 240 | total loss : Tensor("add_5853:0", shape=(), dtype=float32)
epochs 5 | Steps 260 | total loss : Tensor("add_5873:0", shape=(), dtype=float32)
epochs 5 | Steps 280 | total loss : Tensor("add_5893:0", shape=(), dtype=float32)
epochs 5 | Steps 300 | total loss : Tensor("add_5913:0", shape=(), dtype=float32)


epochs 5 | Steps 320 | total loss : Tensor("add_5932:0", shape=(), dtype=float32)
epochs 5 | Steps 340 | total loss : Tensor("add_5952:0", shape=(), dtype=float32)
epochs 5 | Steps 360 | total loss : Tensor("add_5972:0", shape=(), dtype=float32)
epochs 5 | Steps 380 | total loss : Tensor("add_5992:0", shape=(), dtype=float32)
epochs 5 | Steps 400 | total loss : Tensor("add_6012:0", shape=(), dtype=float32)


epochs 5 | Steps 440 | total loss : Tensor("add_6051:0", shape=(), dtype=float32)
epochs 5 | Steps 460 | total loss : Tensor("add_6071:0", shape=(), dtype=float32)
epochs 5 | Steps 480 | total loss : Tensor("add_6091:0", shape=(), dtype=float32)
epochs 5 | Steps 500 | total loss : Tensor("add_6111:0", shape=(), dtype=float32)
epochs 5 | Steps 520 | total loss : Tensor("add_6131:0", shape=(), dtype=float32)


epochs 5 | Steps 540 | total loss : Tensor("add_6150:0", shape=(), dtype=float32)
epochs 5 | Steps 560 | total loss : Tensor("add_6170:0", shape=(), dtype=float32)
epochs 5 | Steps 580 | total loss : Tensor("add_6190:0", shape=(), dtype=float32)
epochs 5 | Steps 600 | total loss : Tensor("add_6210:0", shape=(), dtype=float32)
epochs 5 | Steps 620 | total loss : Tensor("add_6230:0", shape=(), dtype=float32)




In [31]:
model.save('haydn_model.h5')